## Back On Importing My Bullshit

In [22]:
import pandas as pd
import re

from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist
from nltk.corpus import stopwords

from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

## Reading in the CSV after lipstick encoding this pig

In [23]:
df = pd.read_csv('data/tweets.csv')
df = df.dropna(axis=0)
df['Class'] = df.is_there_an_emotion_directed_at_a_brand_or_product

unique_emotions = df.is_there_an_emotion_directed_at_a_brand_or_product.value_counts().index
for emotion in unique_emotions:
    if emotion == "Positive emotion":
        df.Class.replace(to_replace=emotion, value=1, inplace=True)
    elif emotion == "Negative emotion":
        df.Class.replace(to_replace=emotion, value=2, inplace=True)
    else:
        df.Class.replace(to_replace=emotion, value=0, inplace=True)

## Cleaning up the Unexpected Virtue of Ignorance Column Names

In [24]:
df.rename(columns={'tweet_text': 'Tweet'}, inplace=True)
df.rename(columns={'emotion_in_tweet_is_directed_at': 'Focus'}, inplace=True)
df.rename(columns={'is_there_an_emotion_directed_at_a_brand_or_product': 'Emotion'}, inplace=True)

In [25]:
df.Class.value_counts()

1    2672
2     519
0     100
Name: Class, dtype: int64

In [6]:
df

,Tweet,Focus,Emotion,Target
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,2
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,1
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,1
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,2
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,1
...,...,...,...,...
9077,@mention your PR guy just convinced me to swit...,iPhone,Positive emotion,1
9079,&quot;papyrus...sort of like the ipad&quot; - ...,iPad,Positive emotion,1
9080,Diller says Google TV &quot;might be run over ...,Other Google product or service,Negative emotion,2
9085,I've always used Camera+ for my iPhone b/c it ...,iPad or iPhone App,Positive emotion,1


## Reading a totally random tweet at a totally random number

In [ ]:
num = 200
print(df.iloc[num])

## Creating a list of Hashtags to see if there is anything

In [36]:
list_hashtag = []
for num in range(0, df.shape[0]):
    temp = []
    temp = re.findall('#[a-zA-Z0-9]+', df.Tweet.iloc[num])
    temp = list(set(temp))
    if len(temp) > 0:
        list_hashtag.extend(temp)

In [19]:
list_hashtag

['#RISE',
 '#SXSW',
 '#SXSW',
 '#iPad',
 '#SXSW',
 '#sxsw',
 '#SXSW',
 '#googleio',
 '#android',
 '#CTIA',
 '#SXSW',
 '#hollergram',
 '#sxsw',
 '#sxsw',
 '#fail',
 '#sxsw',
 '#SXSW',
 '#SXSW',
 '#SXSW',
 '#sxsw',
 '#hollergram',
 '#sxsw',
 '#iPhone',
 '#SXSW',
 '#SXSW',
 '#SXSW',
 '#sxsw',
 '#SXSW',
 '#sxsw',
 '#Android',
 '#SXSW',
 '#SXSW',
 '#iPad',
 '#SMCDallas',
 '#ChevySMC',
 '#SXSW',
 '#hollergram',
 '#sxsw',
 '#sxsw',
 '#austin',
 '#4sq3',
 '#KeepAustinWeird',
 '#SXSW',
 '#android',
 '#sxsw',
 '#hollergram',
 '#sxsw',
 '#hollergram',
 '#SXSW',
 '#ipad2',
 '#pcma',
 '#sxsw',
 '#eventprofs',
 '#engage365',
 '#SXSW',
 '#Social',
 '#Google',
 '#Circles',
 '#sxsw',
 '#IPad2',
 '#SmartCover',
 '#apple',
 '#SXSW',
 '#SXSW',
 '#sxsw',
 '#sxsw',
 '#bettercloud',
 '#SXSW',
 '#Chaos',
 '#iPad2',
 '#sxsw',
 '#OMFG',
 '#Smile',
 '#sxsw',
 '#sxsw',
 '#sxswi',
 '#SXSW',
 '#sxsw',
 '#sxsw',
 '#fail',
 '#Sxsw',
 '#sxsw',
 '#sxsw',
 '#SXSW',
 '#sxwsi',
 '#sxsw',
 '#sxsh',
 '#Speech',
 '#sxswh',
 

In [28]:
X = df.Tweet
y = df.Class
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [32]:
cv_dict = {}
def reportcrossval(model, model_name, X, Y):
    cv_mean = cross_val_score(model, X, Y).mean()
    cv_dict[model_name] = cv_mean
    print(f"The cv mean of {model_name} is {cv_mean}") 

In [33]:
dummythic = Pipeline([('dummy', DummyClassifier(strategy='most_frequent'))], verbose=True)

In [34]:
reportcrossval(dummythic, 'Dummy', X_train, y_train)

[Pipeline] ............. (step 1 of 1) Processing dummy, total=   0.0s
[Pipeline] ............. (step 1 of 1) Processing dummy, total=   0.0s
[Pipeline] ............. (step 1 of 1) Processing dummy, total=   0.0s
[Pipeline] ............. (step 1 of 1) Processing dummy, total=   0.0s
[Pipeline] ............. (step 1 of 1) Processing dummy, total=   0.0s
The cv mean of Dummy is 0.814019758398962
